In [1]:
# 1) Prepara datos (año, pivot por país y estándar de tecnologías)
import pandas as pd

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]
for c in year_cols: df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
f = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()
YEAR = [c for c in year_cols if f[c].notna().any()][-1]

tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

# Pivot por país (1 punto = país) para el scatter
idx = pd.MultiIndex.from_product([f["Country"].dropna().unique(),
                                  sorted(f["Technology_std"].dropna().unique())],
                                 names=["Country","Technology_std"])
grid = (f.groupby(["Country","Technology_std"])[YEAR].sum(min_count=1)
          .reindex(idx, fill_value=0.0).reset_index())
piv = (grid.pivot(index="Country", columns="Technology_std", values=YEAR)
          .fillna(0.0).reset_index())

for c in ["Solar","Wind","Hydro","Bio","Fossil"]:
    if c not in piv.columns: piv[c] = 0.0
piv["Total"] = piv.drop(columns=["Country"]).sum(axis=1, numeric_only=True)
piv["DominantTech"] = piv[["Solar","Wind","Hydro","Bio","Fossil"]].idxmax(axis=1)

# Para que no se amontonen, quitamos (0,0) solo del SCATTER:
piv_viz = piv[(piv["Solar"]>0) | (piv["Wind"]>0)].copy()
print(f"Año: {YEAR} | países con Solar/Wind>0: {len(piv_viz)}  | filas totales CSV: {len(f)}")

Año: F2023 | países con Solar/Wind>0: 82  | filas totales CSV: 570


In [2]:
# OG DATASET --- Prepare Renewable Energy dataset for ScatterBrush ---
import pandas as pd
import re

# 1) Load + filter valid ISO2
csv_path = "Renewable_Energy.csv"
f = pd.read_csv(csv_path)
iso2_clean = f["ISO2"].astype(str).str.strip()
f = f[iso2_clean.ne("") & f["ISO2"].notna()].copy()

# 2) Detect available year columns dynamically
year_cols = [c for c in f.columns if re.fullmatch(r"F\d{4}", c)]
year_cols = sorted(year_cols)

# 3) Standardize Technology names before combining
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

# Extract MW/GWh abbreviation
def extract_abbr(unit: str):
    if not isinstance(unit, str):
        return None
    m = re.search(r"\b(MW|GWh)\b", unit, flags=re.IGNORECASE)
    return m.group(1) if m else None

f["UnitAbbr"] = f["Unit"].apply(extract_abbr)
f = f[f["UnitAbbr"].notna()].copy()

# 4) Create TechUnit using standardized names + abbrev
f["TechUnit"] = f["Technology_std"].astype(str).str.strip() + " (" + f["UnitAbbr"] + ")"

# Convert year columns to numeric and drop rows with no data
for c in year_cols:
    f[c] = pd.to_numeric(f[c], errors="coerce")
f = f.dropna(subset=year_cols, how="all")

# 5) Keep only relevant columns
keep_cols = ["ObjectId", "Country", "TechUnit"] + year_cols
df_scatter = f.loc[:, keep_cols].reset_index(drop=True)

# 6) Auto-detect TechUnit options for the scatter widget
tech_options = sorted(df_scatter["TechUnit"].unique().tolist())

# 7) Create XY_var* kwargs automatically
xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options)}

# Optional: get min/max year for slider
year_min = min(int(c[1:]) for c in year_cols)
year_max = max(int(c[1:]) for c in year_cols)

print(f"TechUnits detected: {len(tech_options)} → {tech_options[:5]}...")
print(f"Years available: {year_min}–{year_max}")
df_scatter.head()

TechUnits detected: 10 → ['Bio (GWh)', 'Bio (MW)', 'Fossil (GWh)', 'Fossil (MW)', 'Hydro (GWh)']...
Years available: 2000–2022


,ObjectId,Country,TechUnit,F2000,F2001,F2002,F2003,F2004,F2005,F2006,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
0,11,"Afghanistan, Islamic Rep. of",Fossil (GWh),31.640,31.640,31.640,110.101,270.930,270.930,270.930,...,207.375,170.138,146.663,148.318,163.618,192.930,176.710,151.790,257.780,234.880
1,12,"Afghanistan, Islamic Rep. of",Fossil (MW),29.725,29.725,29.725,37.033,52.013,52.013,52.013,...,236.661,236.661,236.661,236.661,236.661,236.661,276.661,276.661,276.661,276.661
2,13,"Afghanistan, Islamic Rep. of",Hydro (GWh),457.939,457.939,457.949,458.324,458.651,461.584,659.959,...,854.829,967.683,1000.571,1024.908,1042.548,881.785,1220.959,988.256,849.791,737.882
3,14,"Afghanistan, Islamic Rep. of",Hydro (MW),191.503,191.503,191.506,191.619,191.718,192.601,195.647,...,276.697,280.146,283.595,329.044,348.993,348.993,348.993,348.993,356.608,459.138
4,15,"Afghanistan, Islamic Rep. of",Solar (GWh),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.643,32.319,33.319,35.551,39.347,38.949,55.848,65.991,78.786,93.876


In [3]:
# OG DATASET --- Build WIDE Energy table: ObjectId; Country; Continent; <TechUnit>__FYYYY for all years ---

import re
import pandas as pd

# 1) Load + ISO2 filter
csv_path = "Renewable_Energy.csv"
f = pd.read_csv(csv_path)
iso2_clean = f["ISO2"].astype(str).str.strip()
f = f[iso2_clean.ne("") & f["ISO2"].notna()].copy()

# 2) Detect FYYYY year columns
year_cols = sorted([c for c in f.columns if re.fullmatch(r"F\d{4}", c)])
for c in year_cols:
    f[c] = pd.to_numeric(f[c], errors="coerce")

# 3) Standardize Technology + extract MW/GWh
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

def extract_abbr(unit: str):
    if not isinstance(unit, str):
        return None
    m = re.search(r"\b(MW|GWh)\b", unit, flags=re.IGNORECASE)
    return m.group(1).upper() if m else None

f["UnitAbbr"] = f["Unit"].apply(extract_abbr)
f = f[f["UnitAbbr"].notna()].copy()

# 4) TechUnit = "<Tech> (<MW|GWh>)"
f["TechUnit"] = f["Technology_std"].astype(str).str.strip() + " (" + f["UnitAbbr"] + ")"

# 5) Continent from ISO2 using libraries (comprehensive)

def add_continent_from_iso2(df: pd.DataFrame) -> pd.DataFrame:
    s = df["ISO2"].astype(str).str.strip().str.upper().replace({"": None, "NA": None, "NAN": None})

    # Preferred: country_converter
    try:
        import country_converter as coco
        cc = coco.CountryConverter()
        # coco returns a list; convert back to a Series aligned to index
        cont_list = cc.convert(names=s.tolist(), src="ISO2", to="continent", not_found=None)
        cont_series = pd.Series(cont_list, index=df.index)
        df["Continent"] = cont_series.fillna("Unknown")
        return df
    except Exception:
        pass

    # Fallback: pycountry_convert
    try:
        from pycountry_convert import (
            country_alpha2_to_continent_code as a2_to_cc,
            convert_continent_code_to_continent_name as cc_to_name,
        )
        def conv(a2):
            try:
                return cc_to_name(a2_to_cc(a2))
            except Exception:
                return None
        cont_series = s.map(conv).fillna("Unknown")
        df["Continent"] = cont_series
        return df
    except Exception:
        # Last resort: mark unknown (so pipeline doesn’t break)
        df["Continent"] = "Unknown"
        return df

# Apply
f = add_continent_from_iso2(f)

# 6) Ensure ObjectId exists (if not in CSV, synthesize)
if "ObjectId" not in f.columns:
    f["ObjectId"] = range(1, len(f) + 1)

# 7) Build WIDE table: index = (ObjectId, Country, Continent); columns = "<TechUnit>__FYYYY"
#   a) aggregate per country+techunit (sum over duplicates)
agg = f.groupby(["Country","Continent","TechUnit"], as_index=False)[year_cols].sum()    #! DEBUG REMOVED: "ObjectId",
#   b) long -> add combined column name "<TechUnit>__FYYYY"
long = agg.melt(id_vars=["Country","Continent","TechUnit"],                             #! DEBUG REMOVED: "ObjectId",
                value_vars=year_cols, var_name="YearCol", value_name="Value")
long["col"] = long["TechUnit"] + "__" + long["YearCol"]
#   c) wide pivot
wide = (long.pivot_table(index=["Country","Continent"], columns="col",                  #! DEBUG REMOVED: "ObjectId",
                         values="Value", aggfunc="sum")
            .reset_index()
            .fillna(0.0))

# 8) Output objects you’ll use later
df_wide = wide  # columns: ObjectId, Country, Continent, <TechUnit>__FYYYY, ...
tech_options = sorted(f["TechUnit"].unique().tolist())  # all TechUnits (MW+GWh)
xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options)}  # for widget call

print(f"Wide shape: {df_wide.shape}  | TechUnits: {len(tech_options)}  | Years: {len(year_cols)}")

# Consistent color palette for continents
continent_colors = {
    "Asia": "#1f77b4",
    "Europe": "#ff7f0e",
    "Africa": "#2ca02c",
    "Oceania": "#d62728",
    "America": "#9467bd",
}

df_wide

Wide shape: (219, 232)  | TechUnits: 10  | Years: 23


col,Country,Continent,Bio (GWH)__F2000,Bio (GWH)__F2001,Bio (GWH)__F2002,Bio (GWH)__F2003,Bio (GWH)__F2004,Bio (GWH)__F2005,Bio (GWH)__F2006,Bio (GWH)__F2007,...,Wind (MW)__F2013,Wind (MW)__F2014,Wind (MW)__F2015,Wind (MW)__F2016,Wind (MW)__F2017,Wind (MW)__F2018,Wind (MW)__F2019,Wind (MW)__F2020,Wind (MW)__F2021,Wind (MW)__F2022
0,"Afghanistan, Islamic Rep. of",Unknown,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.1,0.1,0.1,0.1,0.4,0.4,0.40,0.4,0.4,0.4
1,Albania,Unknown,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,Algeria,Unknown,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,10.2,10.2,10.2,10.2,10.0,10.00,10.0,10.0,10.0
3,American Samoa,Unknown,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Andorra, Principality of",Unknown,0.000,0.000,0.000,0.000,1.000,1.000,4.000,19.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,Vietnam,Unknown,300.886,293.138,342.400,337.094,312.986,298.974,334.390,347.934,...,53.0,53.0,136.0,160.2,204.7,236.7,374.55,518.0,4118.0,5065.0
215,West Bank and Gaza,Unknown,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
216,"Yemen, Rep. of",Unknown,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
217,Zambia,Unknown,39.013,39.013,39.013,39.013,39.013,39.366,39.013,38.660,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [4]:
from Isea.scatter import ScatterBrush
import re

# Year range
years_int = sorted({int(m.group(1)) for c in df_wide.columns
                    for m in [re.search(r"__F(\d{4})$", str(c))] if m})
year_min, year_max = years_int[0], years_int[-1]


xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options, start=1)}

rows = df_wide.to_dict("records")   # full table; widget remaps per year

w_scatter = ScatterBrush(
    data=rows,
    x=tech_options[1], y=tech_options[3],
    color="Continent",
    colorMap=continent_colors,
    label="Country",      # ✅ display name
    key="Country",        # ✅ stable unique id (one row per country now)
    width=1200, height=500,
    panel_position="right", panel_width=320, panel_height=220,
    YearMin=year_min, YearMax=year_max,
    **xy_kwargs,
)
display(w_scatter)

ScatterBrush(data=[{'Country': 'Afghanistan, Islamic Rep. of', 'Continent': 'Unknown', 'Bio (GWH)__F2000': 0.0…

In [5]:
# --- Minimal linked view: second scatter shows only the selected countries ---

import pandas as pd
from Isea.scatter import ScatterBrush

# Pull current axis/fields from the first widget (robust to either attr or .options)
opts = getattr(w_scatter, "options", {}) or {}
x_col = getattr(w_scatter, "x", None) or opts.get("x")
y_col = getattr(w_scatter, "y", None) or opts.get("y")
label_col = getattr(w_scatter, "label", None) or opts.get("label") or "Country"
color_col = getattr(w_scatter, "color", None) or opts.get("color") or "Continent"
key_col   = getattr(w_scatter, "key", None)   or opts.get("key")   or "Country"


# print(opts, x_col, y_col, label_col, color_col, key_col)
# XY_var* and year range from earlier cells
xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options, start=1)}

# Empty second widget (data will be filled on first selection)
w_scatter_sel = ScatterBrush(
    data=pd.DataFrame([], columns=[label_col, color_col, key_col]).to_dict("records"),
    x=x_col, y=y_col,
    color=color_col,colorMap=continent_colors, 
    label=label_col, key=key_col,
    width=900, height=450, panel_position="right", panel_height=160,
    # title="Linked view (selection from first scatter)",
    YearMin=year_min, YearMax=year_max,
    **xy_kwargs,
)
display(w_scatter_sel)

# Keep a df for downstream cells
df_selected = pd.DataFrame(columns=df_wide.columns)


def _link_selection_to_second(change):
    global df_selected
    sel = change.get("new") or {}
    rows = sel.get("rows") or []

    # countries from the selection (row has Country or fallback to label/key)
    countries = [r.get("Country") or r.get(label_col) or r.get(key_col) for r in rows]
    countries = [c for c in countries if isinstance(c, str)]

    # Update df_selected (full wide rows so year/XY remain interactive)
    if countries:
        df_selected = df_wide[df_wide["Country"].isin(countries)].copy()
    else:
        df_selected = pd.DataFrame(columns=df_wide.columns)

    # Keep linked widget in sync:
    #  - data: full rows (so its slider & XY pills work)
    #  - axes: mirror current axes from the first widget
    cur_x = getattr(w_scatter, "x", None) or opts.get("x")
    cur_y = getattr(w_scatter, "y", None) or opts.get("y")
    w_scatter_sel.data = (
        df_selected.assign(key=lambda d: d["Country"], label=lambda d: d["Country"])
                   .to_dict("records")
    )
    if cur_x and cur_y:
        w_scatter_sel.x = cur_x
        w_scatter_sel.y = cur_y

    # optional: clear selection inside the second chart each update
    w_scatter_sel.selection = {"type": None, "keys": [], "rows": [], "epoch": int(__import__("time").time()*1000)}

# Wire first -> second (live updates)
w_scatter.observe(_link_selection_to_second, names="selection")

C:\Users\alexa\AppData\Local\Temp\ipykernel_3000\3331563500.py:21: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  data=pd.DataFrame([], columns=[label_col, color_col, key_col]).to_dict("records"),


ScatterBrush(options={'x': 'Bio (MW)', 'y': 'Fossil (MW)', 'label': 'Country', 'color': 'Continent', 'key': 'C…

In [6]:
import pandas as pd
from Isea import ParallelEnergy

df = pd.read_csv("Isea/Energy_clean.csv")

years = [c for c in df.columns if c.startswith("F")]
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])


In [ ]:
w = ParallelEnergy(
    df, years,
    tech_col="Technology_std",
    label_col="Country",
    dims=("Fossil", "Solar", "Hydro", "Wind", "Bio"),
    year_start="F2023",
    width=1100, height=560,
    margin=dict(left=50, right=180, top=80, bottom=36),  # ajusta si quieres
    panel_position="bottom",       # "right" o "bottom"
    panel_width=320, panel_height=220,
    log_axes=False, normalize=False, reorder=True
)
w


ParallelEnergy(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008'…

In [8]:
# w.show_selection()

In [9]:
from IPython.display import display, clear_output
import ipywidgets as ipw

out = ipw.Output()
display(out)

def on_sel(change):
    with out:
        clear_output(wait=True)
        df_sel = w.selection_df()
        print(f"point selection — {len(df_sel)} points")
        display(df_sel)

# Escucha cambios del widget
w.observe(on_sel, "selection")


Output()

In [10]:
# w2 = w.new_from_selection(
#     width=1000, height=520,
#     margin=dict(l=48, r=160, t=72, b=32),
#     panel_position="bottom",
#     panel_height=200
# )
# w2


In [11]:
import inspect
from Isea.radial_stacked_bar import RadialStackedBar

# Ver la firma de la función
print(inspect.signature(RadialStackedBar.__init__))

# Ver si year_cols está en los parámetros
params = inspect.signature(RadialStackedBar.__init__).parameters
print("year_cols en parámetros:", "year_cols" in params)

(self, df: pandas.core.frame.DataFrame, group_col: str, category_col: str, year_cols: Optional[List[str]] = None, year_start: Optional[str] = None, agg: str = 'sum', width: int = 900, height: int = 900, inner_radius: int = 200, pad_angle: float = 0.015, color_scheme: str = 'schemeSpectral', sort_on_click: bool = True, title: Optional[str] = None, custom_colors: Optional[List[str]] = None, **kwargs)
year_cols en parámetros: True


In [12]:
# ========== PRIMERO: Preparar datos (IGUAL A LA CELDA 1) ==========
import pandas as pd
from Isea import RadialStackedBar

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

# Convertir a numéricas
for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Filtrar por indicador y unidad
IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

# IMPORTANTE: Estandarizar tecnologías
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

print(f"✅ Datos preparados: {len(df)} filas")
print(f"✅ Columnas: {df.columns.tolist()}")
print(f"✅ Tecnologías únicas: {df['Technology_std'].unique()}")

# ========== AHORA: Crear RadialStackedBar ==========
radial = RadialStackedBar(
    df=df,
    group_col="Country",
    category_col="Technology_std",
    year_cols=year_cols,
    year_start="F2023",
    width=900,
    height=900,
    inner_radius=200,
    title="⚡ Capacidad Instalada",
    custom_colors=["#d73027","#f46d43","#fdae61","#fee08b","#e6f598","#abdda4","#66c2a5","#3288bd"]
)

display(radial)
print("✅ Widget creado exitosamente!")

✅ Datos preparados: 570 filas
✅ Columnas: ['ObjectId', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Technology', 'Energy_Type', 'Unit', 'Source', 'CTS_Name', 'CTS_Code', 'CTS_Full_Descriptor', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023', 'Technology_std']
✅ Tecnologías únicas: ['Fossil' 'Hydro' 'Bio' 'Solar' 'Wind']


RadialStackedBar(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F200…

✅ Widget creado exitosamente!


In [13]:
print("Columns:", df.columns.tolist())
print("Has Technology_std:", "Technology_std" in df.columns)
print("Year cols:", year_cols)
print("Sample data:\n", df[["Country", "Technology_std", year_cols[0]]].head())

Columns: ['ObjectId', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Technology', 'Energy_Type', 'Unit', 'Source', 'CTS_Name', 'CTS_Code', 'CTS_Full_Descriptor', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023', 'Technology_std']
Has Technology_std: True
Year cols: ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023']
Sample data:
                         Country Technology_std     F2000
0  Afghanistan, Islamic Rep. of         Fossil    29.725
1  Afghanistan, Islamic Rep. of          Hydro   191.503
2                       Albania          Hydro  1453.000
3                       Algeria         Fossil  5846.210
4                       Algeria          Hydro   276.600


In [14]:
# Fuerza recarga
import sys
for mod in list(sys.modules.keys()):
    if 'Isea' in mod:
        del sys.modules[mod]

# Reimporta
from Isea import RadialStackedBar
from IPython.display import display

# Preparar datos (mismo que antes)
import pandas as pd

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])
print("_esm path:", RadialStackedBar._esm)
print("Existe:", Path(RadialStackedBar._esm).exists())

# Si no existe, dinos y crearemos el archivo
print("\nContenido esperado del archivo:")
import inspect
print(inspect.getfile(RadialStackedBar))

_esm path: // RadialStackedBar ESM module para anywidget

export function render({ model, el }) {
  // ========== Crear estructura HTML ==========
  const container = document.createElement("div");
  container.style.cssText = `
    display: flex;
    flex-direction: column;
    gap: 12px;
    padding: 16px;
  `;

  // Título
  const title = document.createElement("div");
  title.style.cssText = `
    font-size: 16px;
    font-weight: 600;
    color: #1f2937;
  `;
  container.appendChild(title);

  // Barra de controles
  const controls = document.createElement("div");
  controls.style.cssText = `
    display: flex;
    gap: 12px;
    align-items: center;
    padding: 8px;
    background: #f9fafb;
    border-radius: 8px;
  `;

  const yearLabel = document.createElement("span");
  yearLabel.textContent = "Año:";
  yearLabel.style.cssText = `font-weight: 500; color: #374151;`;
  controls.appendChild(yearLabel);

  const slider = document.createElement("input");
  slider.type = "range";
  

NameError: name 'Path' is not defined

In [ ]:
# ========== PREPARACIÓN AGRESIVA DE DATOS ==========
import pandas as pd
from Isea import RadialStackedBar

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

# Convertir a numéricas
for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Filtrar por indicador y unidad
IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

# Estandarizar tecnologías
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

# ========== Filtrar solo tecnologías estándar ==========
df_clean = df[
    df["Technology_std"].isin(["Fossil", "Solar", "Wind", "Hydro", "Bio"])
].copy()

# ========== NUEVO: Agrupar por país y tecnología ==========
df_grouped = df_clean.groupby(["Country", "Technology_std"])[year_cols].sum().reset_index()

# ========== NUEVO: Filtrar países con TODAS las tecnologías ==========
# Contar cuántas tecnologías tiene cada país
tech_count = df_grouped.groupby("Country")["Technology_std"].nunique()

# Mantener solo países con LAS 5 TECNOLOGÍAS
required_techs = 5  # Fossil, Solar, Wind, Hydro, Bio
complete_countries = tech_count[tech_count == required_techs].index.tolist()

print(f"📊 Países totales: {df_grouped['Country'].nunique()}")
print(f"✅ Países con 5 tecnologías: {len(complete_countries)}")

# Filtrar
df_grouped = df_grouped[df_grouped["Country"].isin(complete_countries)].copy()

# ========== NUEVO: Eliminar filas donde TODOS los años son 0 ==========
df_grouped["Total"] = df_grouped[year_cols].sum(axis=1)
df_grouped = df_grouped[df_grouped["Total"] > 0].drop(columns=["Total"])

# ========== NUEVO: Filtrar por año más reciente con datos ==========
# Solo mantener países con datos en F2023 (año más reciente)
last_year = "F2023"
df_grouped = df_grouped[df_grouped[last_year] > 0].copy()

print(f"✅ Datos finales después de filtrar: {len(df_grouped)} filas")
print(f"✅ Países únicos: {df_grouped['Country'].nunique()}")
print(f"✅ Tecnologías: {df_grouped['Technology_std'].unique()}")

# ========== Crear RadialStackedBar ==========
radial = RadialStackedBar(
    df=df_grouped,
    group_col="Country",
    category_col="Technology_std",
    year_cols=year_cols,
    year_start="F2023",
    width=900,
    height=900,
    inner_radius=200,
    title="⚡ Capacidad Instalada (MW) - Países Completos",
    custom_colors=["#d73027","#f46d43","#fdae61","#fee08b","#e6f598","#abdda4","#66c2a5","#3288bd"]
)

display(radial)
print("✅ Widget creado con datos limpios!")

In [ ]:
# ========== WorldRenewable Widget (AnyWidget Version) ==========
from Isea import WorldRenewable
from IPython.display import display
import pandas as pd

# Prepare data with Technology_std column
df_world = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df_world.columns if c.startswith("F")]
for c in year_cols: 
    df_world[c] = pd.to_numeric(df_world[c], errors="coerce")

# Add Technology_std mapping
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df_world["Technology_std"] = df_world["Technology"].map(tech_map).fillna(df_world["Technology"])

# Create the widget
world_widget = WorldRenewable(
    df=df_world,
    year_cols=year_cols,
    country_col="Country",
    iso3_col="ISO3",
    tech_col="Technology_std",
    start_year="F2023",  # Start at the most recent year
    width=1200,
    height=660,
    normalize=False
)

display(world_widget)

# Add observer to track selection
def on_world_selection(change):
    sel = change["new"]
    if sel and sel.get("iso3"):
        year = sel.get('year', 'N/A')
        value = sel.get('value')
        value_str = f"{value:.1%}" if value is not None else "N/A"
        print(f"Selected: {sel['name']} ({sel['iso3']}) - Year {year} - Renewable: {value_str}")

world_widget.observe(on_world_selection, names="selection")

Final code parallel

In [ ]:
import importlib, Isea
importlib.reload(Isea)

import pandas as pd
from Isea import EnergyQuad

df = pd.read_csv("Isea/Energy_clean.csv")

years = [c for c in df.columns if c.startswith("F")]
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

dash = EnergyQuad(
    df, years,
    tech_col="Technology_std", label_col="Country",
    dims=("Fossil","Solar","Hydro","Wind","Bio"),
    year_start="F2023",
    width=1180, left_width=720,   # balancea columnas
    left_height=460, table_height=190,
    right_width=440, insight_height=230, mini_height=260,
    log_axes=False, normalize=False, reorder=True
)
dash



In [ ]:
dash.selection_df()


In [ ]:
import pandas as pd
#import pycountry

# --- Load the EV dataset ---
df_ev = pd.read_csv("Global_EV_clean.csv")

# Rename columns for clarity
df_ev = df_ev.rename(columns={
    "region": "Country",
    "powertrain": "Technology",
    "year": "Year",
    "value": "Value"
})

# Keep only the relevant columns
df_ev = df_ev[["Country", "Technology", "Year", "Value"]]

# Pivot to wide format with FYYYY columns
df_wide = (
    df_ev
    .pivot_table(index=["Country", "Technology"], columns="Year", values="Value", aggfunc="sum")
    .reset_index()
)
df_wide.columns = ["Country", "Technology"] + [f"F{int(y)}" for y in df_wide.columns[2:]]

# Temporary ISO3 placeholder
df_wide["ISO3"] = df_wide["Country"].str[:3].str.upper()

# Reorder columns
cols = ["Country", "ISO3", "Technology"] + [c for c in df_wide.columns if c.startswith("F")]
df_wide = df_wide[cols]

# --- Save cleaned version for the widget ---
df_wide.to_csv("Isea/Global_EV_clean_ready.csv", index=False)

print("✅ Saved: Isea/Global_EV_clean_ready.csv")
df_wide.head()


PermissionError: [Errno 13] Permission denied: 'Isea/Global_EV_clean_ready.csv'

In [ ]:
import pandas as pd

# Load the cleaned EV dataset
df_ev = pd.read_csv("Isea/Global_EV_clean_ready.csv")

# Manual mapping for common countries
iso_map = {
    "United States": "USA", "United Kingdom": "GBR", "Russia": "RUS", "South Korea": "KOR",
    "North Korea": "PRK", "Iran": "IRN", "Viet Nam": "VNM", "Czech Republic": "CZE",
    "Democratic Republic of the Congo": "COD", "Congo": "COG", "Ivory Coast": "CIV",
    "Eswatini": "SWZ", "Tanzania": "TZA", "Syria": "SYR", "Laos": "LAO",
    "Moldova": "MDA", "Bolivia": "BOL", "Venezuela": "VEN", "Taiwan": "TWN",
    "Kosovo": "XKX", "Palestine": "PSE", "Hong Kong": "HKG", "Macau": "MAC",
    "Myanmar": "MMR", "Brunei Darussalam": "BRN", "Cape Verde": "CPV",
    "East Timor": "TLS", "North Macedonia": "MKD", "Serbia": "SRB",
    "Slovakia": "SVK", "South Sudan": "SSD", "Bosnia and Herzegovina": "BIH",
    "United Arab Emirates": "ARE", "Dominican Republic": "DOM", "Trinidad and Tobago": "TTO",
    "Papua New Guinea": "PNG", "New Zealand": "NZL", "Australia": "AUS",
    "Canada": "CAN", "Mexico": "MEX", "Japan": "JPN", "China": "CHN",
    "India": "IND", "Brazil": "BRA", "Argentina": "ARG", "Chile": "CHL",
    "Peru": "PER", "Colombia": "COL", "Ecuador": "ECU", "Uruguay": "URY",
    "Paraguay": "PRY", "Nigeria": "NGA", "Kenya": "KEN", "Egypt": "EGY",
    "Morocco": "MAR", "Algeria": "DZA", "South Africa": "ZAF", "Turkey": "TUR",
    "Saudi Arabia": "SAU", "Israel": "ISR", "Pakistan": "PAK", "Indonesia": "IDN",
    "Malaysia": "MYS", "Philippines": "PHL", "Thailand": "THA", "Singapore": "SGP",
    "Vietnam": "VNM", "Poland": "POL", "France": "FRA", "Germany": "DEU",
    "Italy": "ITA", "Spain": "ESP", "Portugal": "PRT", "Netherlands": "NLD",
    "Belgium": "BEL", "Switzerland": "CHE", "Sweden": "SWE", "Norway": "NOR",
    "Finland": "FIN", "Denmark": "DNK", "Austria": "AUT", "Hungary": "HUN",
    "Greece": "GRC", "Romania": "ROU", "Ukraine": "UKR", "Belarus": "BLR",
}

# Build ISO3 column (manual map + fallback)
df_ev["ISO3"] = df_ev["Country"].map(iso_map)
df_ev["ISO3"] = df_ev["ISO3"].fillna(df_ev["Country"].str[:3].str.upper())

# Check which countries still have no ISO3
missing = df_ev[df_ev["ISO3"].isna()]["Country"].unique()
print("⚠️ Missing ISO3 for:", missing)

# Save updated dataset
df_ev.to_csv("Isea/Global_EV_clean_ready.csv", index=False)
print("✅ Saved updated dataset with ISO3 codes")


⚠️ Missing ISO3 for: []


PermissionError: [Errno 13] Permission denied: 'Isea/Global_EV_clean_ready.csv'

: 

In [ ]:
import pandas as pd

# Load the EV dataset you're using in the widget
df_ev = pd.read_csv("Isea/Global_EV_clean_ready.csv")

# --- 1) Standardize Technology names (case/typo/synonyms) ---
norm = {
    "battery electric vehicle": "BEV",
    "battery electric vehicles": "BEV",
    "bev": "BEV",
    "plug-in hybrid": "PHEV",
    "plug-in hybrid": "PHEV",
    "phev": "PHEV",
    "fuel cell": "FCEV",
    "fcev": "FCEV",
    "internal combustion engine": "ICE",
    "ice": "ICE",
    "other": "Other",
    "others": "Other",
    "total": "Total",
}
def clean_label(x):
    if not isinstance(x, str): return x
    k = x.strip().lower().replace("–","-").replace("—","-")
    return norm.get(k, x.strip())

df_ev["Technology"] = df_ev["Technology"].apply(clean_label)

# --- 2) Ensure year columns exist and numeric ---
year_cols = [c for c in df_ev.columns if isinstance(c,str) and c.startswith("F")]
for c in year_cols:
    df_ev[c] = pd.to_numeric(df_ev[c], errors="coerce")

# --- 3) Decide buckets that actually exist in this file ---
candidates = ["BEV","PHEV","FCEV","Other","ICE","Total"]
present = [t for t in candidates if t in set(df_ev["Technology"].unique())]
# Prefer EV stack + ICE; if Total exists, use it for denominator too
tech_keep = [t for t in ["BEV","PHEV","FCEV","Other","ICE"] if t in present]
if "Total" in present and "Total" not in tech_keep:
    tech_keep.append("Total")

# --- 4) Pick numerator/denominator robustly ---
ev_num = [t for t in ["BEV","PHEV","FCEV"] if t in present]
if "Total" in present:
    share_num = ev_num
    share_den = ["Total"]
else:
    # fall back to all buckets available; avoids zero-denominator
    share_num = ev_num
    share_den = tech_keep.copy()

# Save back (optional) so the rest of your code reads the standardized file
df_ev.to_csv("Isea/Global_EV_clean_ready.csv", index=False)

print("Techs present:", present)
print("tech_keep ->", tech_keep)
print("share_numerator ->", share_num)
print("share_denominator ->", share_den)
print("Years detected:", year_cols[:3], "... total", len(year_cols))


In [ ]:
# --- EV filter widgets: BEV, EV, FCEV, PHEV, fast, slow ---
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load EV dataset used elsewhere (safe to re-read)
df_ev = pd.read_csv("Isea/Global_EV_clean_ready.csv")

# Define checkboxes for each option
options = ["BEV","EV","FCEV","PHEV","fast","slow"]
checkboxes = {opt: widgets.Checkbox(value=False, description=opt) for opt in options}
box = widgets.VBox([widgets.HBox([checkboxes[opt] for opt in options[:3]]), widgets.HBox([checkboxes[opt] for opt in options[3:]])])

out = widgets.Output()

def _filter_ev_df(selected):
    df = df_ev.copy()
    if not selected:
        return df
    parts = []
    for s in selected:
        if s == 'EV':
            parts.append(df[df['Technology'].isin(['BEV','PHEV','FCEV'])])
        elif s in ('BEV','PHEV','FCEV'):
            parts.append(df[df['Technology'] == s])
        elif s in ('fast','slow'):
            # try to detect a charging/speed column; fallback: look for string in any column
            cols = [c for c in df.columns if any(k in c.lower() for k in ('charge','charger','speed','type'))]
            if cols:
                masks = [df[c].astype(str).str.contains(s, case=False, na=False) for c in cols]
                mask = False
                for m in masks:
                    mask = mask | m
                parts.append(df[mask])
            else:
                # no charger info available -> produce empty frame for this part
                parts.append(df.iloc[0:0])
        else:
            parts.append(df.iloc[0:0])
    if parts:
        out_df = pd.concat(parts, ignore_index=True).drop_duplicates().reset_index(drop=True)
    else:
        out_df = df.iloc[0:0]
    return out_df

def _on_change(change=None):
    selected = [k for k,v in checkboxes.items() if v.value]
    with out:
        clear_output(wait=True)
        print("Selected:", selected)
        res = _filter_ev_df(selected)
        print(f"Rows returned: {len(res)}")
        display(res.head(10))

for cb in checkboxes.values():
    cb.observe(_on_change, names='value')

display(widgets.Label("Filter EV dataset (toggle checkboxes):"))
display(box)
display(out)

# Initial display
_on_change()

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from Isea.worldmaplinechart import WorldMapLineChart

# --- Load both datasets ---
df_energy = pd.read_csv("Isea/Energy_clean.csv")
df_ev = pd.read_csv("Isea/Global_EV_clean_ready.csv")

# --- Common setup for both ---
def make_energy_widget():
    year_cols = [c for c in df_energy.columns if c.startswith("F")]
    tech_map = {
        "Hydropower (excl. Pumped Storage)": "Hydro",
        "Solar energy": "Solar",
        "Wind energy": "Wind",
        "Bioenergy": "Bio",
        "Fossil fuels": "Fossil",
    }
    return WorldMapLineChart(
        df_energy,
        year_cols=year_cols,
        country_col="Country",
        iso3_col="ISO3",
        tech_col="Technology",
        tech_keep=["Solar","Wind","Hydro","Bio","Fossil"],
        tech_map=tech_map,
        share_numerator=["Solar","Wind","Hydro","Bio"],
        title="Renewables share of installed capacity by country",
        subtitle="Move the year slider. Click countries to compare.",
        share_label="Renewables share",
    )

def make_ev_widget():
    df_ev2 = pd.read_csv("Isea/Global_EV_clean_ready.csv")
    year_cols = [c for c in df_ev2.columns if isinstance(c,str) and c.startswith("F")]

    # Recompute the same logic quickly to pass into the widget
    present = set(df_ev2["Technology"].unique())
    tech_keep = [t for t in ["BEV","PHEV","FCEV","Other","ICE"] if t in present]
    if "Total" in present and "Total" not in tech_keep:
        tech_keep.append("Total")

    # Force BEV to be plotted as an absolute series in the line chart by
    # setting numerator == denominator == ['BEV'] when BEV exists. This makes
    # the backend set `shares` == BEV values (no division), and the frontend
    # will plot those absolute numbers with appropriate scaling.
    if "BEV" in present:
        share_numerator = ["BEV"]
        share_denominator = ["BEV"]
    else:
        # fallback: if BEV not present, show EV types as share (original behavior)
        share_numerator = [t for t in ["BEV","PHEV","FCEV"] if t in present]
        share_denominator = ["Total"] if "Total" in present else tech_keep.copy()

    return WorldMapLineChart(
        df_ev2,
        year_cols=year_cols,
        country_col="Country",
        iso3_col="ISO3",
        tech_col="Technology",
        tech_keep=tech_keep,
        tech_map={},                       # already normalized
        share_numerator=share_numerator,   # BEV (absolute series if available)
        share_denominator=share_denominator,
        title="BEV count by country",
        subtitle="Line shows BEV counts (absolute) over years.",
        share_label="BEV",                # ensures absolute scaling in frontend
    )


# --- Selector widget ---
dataset_selector = widgets.Dropdown(
    options=["Renewable Energy", "Electric Vehicles"],
    value="Renewable Energy",
    description="Dataset:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="350px"),
)

output = widgets.Output()

def on_select(change):
    with output:
        clear_output(wait=True)
        if change["new"] == "Renewable Energy":
            display(make_energy_widget())
        else:
            display(make_ev_widget())

dataset_selector.observe(on_select, names="value")

display(dataset_selector, output)

# Trigger first load
on_select({"new": dataset_selector.value})

In [ ]:
import pandas as pd

# --- 1️⃣ Load EV dataset ---
df_ev = pd.read_csv("Isea/Global_EV_clean_ready.csv")
year_cols = [c for c in df_ev.columns if c.startswith("F")]

# --- 2️⃣ Load or define population data (static millions → individuals) ---
pop_data = [
    ("CHN", 1425), ("IND", 1410), ("USA", 339), ("IDN", 277), ("PAK", 241),
    ("BRA", 216), ("NGA", 223), ("RUS", 144), ("JPN", 125), ("MEX", 128),
    ("DEU", 84), ("FRA", 65), ("GBR", 67), ("ITA", 59), ("CAN", 40),
    ("KOR", 52), ("AUS", 26), ("ESP", 48), ("POL", 38), ("ZAF", 61),
    ("SAU", 36), ("TUR", 86), ("IRN", 86), ("EGY", 112), ("ARG", 46),
    ("COL", 52), ("THA", 71), ("VNM", 100), ("SWE", 10), ("NOR", 5),
    ("CHE", 9), ("NLD", 18), ("BEL", 12), ("AUT", 9), ("FIN", 6),
    ("DNK", 6), ("SGP", 6), ("ISR", 9), ("NZL", 5), ("PRT", 10),
]
pop_df = pd.DataFrame(pop_data, columns=["ISO3", "Pop2023"])
pop_df["Pop2023"] *= 1_000_000

# --- 3️⃣ Aggregate EVs (BEV + PHEV + FCEV) and totals ---
df_ev_sum = (
    df_ev[df_ev["Technology"].isin(["BEV", "PHEV", "FCEV"])]
    .groupby(["Country", "ISO3"], as_index=False)[year_cols]
    .sum()
)
df_ev_sum["Technology"] = "EV_total"

df_total = df_ev[df_ev["Technology"] == "Total"].copy()

# --- 4️⃣ Merge population ---
df_merge = df_total.merge(df_ev_sum, on=["Country", "ISO3"], suffixes=("_total", "_ev"))
df_merge = df_merge.merge(pop_df, on="ISO3", how="left")

# --- 5️⃣ Compute EVs per person = EV_share × (Total vehicles ÷ population)
for c in year_cols:
    share = df_merge[f"{c}_ev"] / df_merge[f"{c}_total"]  # EV share
    df_merge[c] = share                                  # EV share per total vehicles
    # alternatively if absolute EV counts: df_merge[c] = df_merge[f"{c}_ev"] / df_merge["Pop2023"]

# choose this ↓ for EVs per person (absolute EV count divided by population):
for c in year_cols:
    df_merge[c] = df_merge[f"{c}_ev"] / df_merge["Pop2023"]

# --- 6️⃣ Final cleanup ---
df_merge = df_merge[["Country", "ISO3"] + year_cols]
df_merge["Technology"] = "EVs_per_capita"
df_merge = df_merge[["Country", "ISO3", "Technology"] + year_cols]

df_merge.to_csv("Isea/Global_EV_percapita.csv", index=False)
print("✅ Saved: Isea/Global_EV_percapita.csv")
print(df_merge.head(5))



In [ ]:
from Isea.worldmaplinechart import WorldMapLineChart
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# --- Energy ---
def make_energy_widget():
    df_energy = pd.read_csv("Isea/Energy_clean.csv")
    year_cols = [c for c in df_energy.columns if c.startswith("F")]
    tech_map = {
        "Hydropower (excl. Pumped Storage)": "Hydro",
        "Solar energy": "Solar",
        "Wind energy": "Wind",
        "Bioenergy": "Bio",
        "Fossil fuels": "Fossil",
    }
    return WorldMapLineChart(
        df_energy,
        year_cols=year_cols,
        country_col="Country",
        iso3_col="ISO3",
        tech_col="Technology",
        tech_keep=["Solar","Wind","Hydro","Bio","Fossil"],
        tech_map=tech_map,
        share_numerator=["Solar","Wind","Hydro","Bio"],
        title="Renewables share of installed capacity by country",
        subtitle="Move the year slider. Click countries to compare.",
        share_label="Renewables share",
    )

def make_ev_percapita_widget():
    df_pc = pd.read_csv("Isea/Global_EV_percapita.csv")
    year_cols = [c for c in df_pc.columns if c.startswith("F")]
    return WorldMapLineChart(
        df_pc,
        year_cols=year_cols,
        country_col="Country",
        iso3_col="ISO3",
        tech_col="Technology",
        tech_keep=["EVs_per_capita"],
        share_numerator=["EVs_per_capita"],
        share_denominator=["EVs_per_capita"],
        title="EVs per Person by Country",
        subtitle="Shows the number of electric vehicles per person (EVs ÷ population).",
        share_label="EVs per person",  # 👈 ensures absolute scaling
    )

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Dropdown selector for all visualization modes
dataset_selector = widgets.Dropdown(
    options=[
        "Renewable Energy",
        "EVs per Capita"   # 👈 new option added
    ],
    value="Renewable Energy",
    description="Dataset:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="350px"),
)

output = widgets.Output()

def on_select(change):
    with output:
        clear_output(wait=True)
        if change["new"] == "Renewable Energy":
            display(make_energy_widget())
        elif change["new"] == "EVs per Capita":
            display(make_ev_percapita_widget())

dataset_selector.observe(on_select, names="value")

display(dataset_selector, output)
on_select({"new": dataset_selector.value})

In [ ]:
import pandas as pd
df = pd.read_csv("Isea/Global_EV_popweighted.csv")
print(df.head(3))
print(df.filter(like="F").max().max())
